In [ ]:
texts = [
    [
        "Give it a try!",
        "Take a moment to try it by yourself, and see how easy it is to work with PDFs in Foxit PDF Editor!",
        "For additional help and instructions: Go to the Discover view",
        "Tap Help",
    ]
]

target_language = "English"

# model ="us.amazon.nova-pro-v1:0"
model="us.amazon.nova-lite-v1:0"

In [ ]:
from langchain_aws import ChatBedrock

chat_model = ChatBedrock(
    model=model,
    region="us-east-1",
    aws_access_key_id="",
    aws_secret_access_key="",
)

In [ ]:
import textwrap

from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)


translate_multi_texts_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            textwrap.dedent(
                """
                You are a helpful assistant that can translate texts to {language}.
                We will provide you a json format with a list of texts to translate.
                
                There are several rules you MUST follow:
                1. Translate the original text as far as possible and do not lose the content. 
                2. Translate the text wherever possible. If this is not possible, return to the original text
                3. Never translate the links, code in the text.
                4. Always return the translated texts in the same order as the input texts.
                5. We will provide you {size} texts to translate. You should generate {size} texts in the target language.
                """
            )
        ),
        HumanMessagePromptTemplate.from_template("Texts that needs to be translated is as follows: \n{texts}"),
    ]
)

translate_multi_texts_structured_output_tool = {
    "name": "translate_texts_result",
    "description": "translate the json format user input into the target language, and return the translated texts",
    "parameters": {
        "type": "object",
        "properties": {
            "translated_texts": {
                "type": "array",
                "items": {"type": "string"},
                "description": "The translated texts",
            },
        },
        "required": ["translated_texts"],
    },
}

In [ ]:
import json


prompts = translate_multi_texts_prompt.format_messages(
    language=target_language,
    size=len(texts),
    texts=json.dumps(texts, ensure_ascii=False, indent=2),
)
res = chat_model.with_structured_output(
    translate_multi_texts_structured_output_tool
).invoke(prompts)
res